# Bitbubble

Simplified ABM model that links the **"How to grow a bubble: A model of myopic adapting agents"** with the **"Leader–follower model for agent based simulation of social collective behavior during egress"** in the bitcoin bubble model space

In [6]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib as mpl
from mesa import Agent, Model
import matplotlib.pyplot as plt
from mesa.time import RandomActivation
%matplotlib inline
#print(mpl.style.available)
mpl.style.use('ggplot')
plt.xkcd()

<matplotlib.pyplot.xkcd.<locals>.dummy_ctx at 0x7f75ad1bb240>

In [5]:
class Investor(Agent):
    """An investor with fixed initial cash and coins."""
    def __init__(self, unique_id, model, C1=1, C2=1, C3=1
                 , alpha=0.95, Omega=2, g=0.02, cash=1.0, coins=1.0):
        super().__init__(unique_id, model)
        self.cash = cash
        self.coins = coins
        self.alpha = alpha
        self.Omega = Omega
        self.g = g
        #TODO: how to generate closed interval random numbers?
        self.c1 = np.random.uniform(0.0, C1)
        self.c2 = np.random.uniform(0.0, C1)
        self.c3 = np.random.uniform(0.0, C3)
        self.w_min = np.random.uniform(0.0, Omega)
        
    def collect_private_info(self):
        return np.random.normal()

In [4]:
class MarketMaker(Model):
    """A model with some number of investors."""
    def __init__(self, N, market_depth):
        self.num_agents = N
        self.lamda = market_depth
        # Create agents
        for i in range(self.num_agents):
            a = Investor(i, self)